In [ ]:
#| default_exp callback.core

# Callback Core
> Foundations for the `Callback` system

In [12]:
#| export
from enum import Enum, auto

In [54]:
#| export
class StrEnum(str, Enum):
    """
    Same as the `StrEnum` available in Python 3.11, returns
    the lower-cased version of the member name.
    """
    @classmethod
    def contains(cls, value) -> bool:
        "Checks if `value` exists in the current `Enum`"
        return value in cls._value2member_map_
    
    def _generate_next_value_(name, start, count, last_values):
        return name.lower()

In [52]:
#| hide
_test_enum = StrEnum("_TestEnum", ("a","B"))
assert _test_enum.a == "a"
assert _test_enum.B == "b"
assert not _test_enum.contains("B")

In [31]:
#| export
class CallbackEvent(StrEnum):
    """
    All possible events for Callbacks
    """
    after_create = auto()
    before_fit = auto()
    before_epoch = auto()
    before_train = auto()
    before_batch = auto()
    after_pred = auto()
    after_loss = auto()
    before_backward = auto()
    after_cancel_backward = auto()
    after_backward = auto()
    before_step = auto()
    after_cancel_step = auto()
    after_step = auto()
    after_cancel_batch = auto()
    after_batch = auto()
    after_cancel_train = auto() 
    after_train = auto()
    before_validate = auto()
    after_cancel_validate = auto()
    after_validate = auto()
    after_cancel_epoch = auto()
    after_epoch = auto()
    after_cancel_fit = auto()
    after_fit = auto()

In [22]:
#| export
class InnerTrainingLoopEvent(StrEnum):
    """
    An Enum containing all of the possible events of the inner training loop
    """
    before_batch = auto()
    after_pred = auto()
    after_loss = auto()
    before_backward = auto()
    after_cancel_backward = auto()
    after_backward = auto()
    before_step = auto()
    after_step = auto()
    after_cancel_batch = auto()
    after_batch = auto()

In [23]:
#| export
class CancelBatchException(Exception):
    """
    An Exception to be used in a `Callback` to know when
    to skip the rest of the current batch and continue to `after_batch`.
    """
    pass

In [23]:
#| export
class CancelTrainException(Exception):
    """
    An Exception to be used in a `Callback` to know when
    to skip the rest of the training section of the current epoch
    and continue to `after_train`.
    """
    pass

In [23]:
#| export
class CancelValidatException(Exception):
    """
    An Exception to be used in a `Callback` to know when
    to skip the rest of the validation section of the current epoch
    and continue to `after_validate`.
    """
    pass

In [23]:
#| export
class CancelEpochException(Exception):
    """
    An Exception to be used in a `Callback` to know when
    to skip the rest of the current epoch and continue 
    to `after_epoch`.
    """
    pass

In [23]:
#| export
class CancelStepException(Exception):
    """
    An Exception to be used in a `Callback` to know when
    to skip the stepping of the optimizer for the 
    current batch.
    """
    pass

In [23]:
#| export
class CancelBackwardException(Exception):
    """
    An Exception to be used in a `Callback` to know when
    to skip the backward pass during the current batch and 
    continue to `after_backward`.
    """
    pass

In [23]:
#| export
class CancelFitException(Exception):
    """
    An Exception to be used in a `Callback` to know when
    to cancel the entire training loop and continue to 
    `after_fit`.
    """
    pass

In [36]:
#| export
class Callback:
    """
    Basic class for handling tweaks of the current training loop through all defined events 
    in the `CallbackEvent` enum.
    
    Class Properties:
    
    - `order` (`int`): The order a `Callback` will be called at
    - `run` (`bool`): Whether the `Callback` should be skipped
    - `run_train` (`bool`): Whether the `Callback` should be skipped during the training stage
    - `run_valid` (`bool`): Whether the `Callback` should be skipped during the validation stage
    """
    order = 0
    run = True
    run_train = True
    run_valid = True
    
    def __init__(self, learn):
        self.learn = learn
        
    def __call__(
        self, 
        event_name:str, # An event defined in `CallbackEvent`
    ):
        if not CallbackEvent.contains(event_name):
            raise ValueError(
                f"Unsupported event {event_name} passed. Please ensure it's one defined in the `CallbackEvent` enum class."
            )
        if (
            not InnerTrainingLoopEvent.contains(event_name)
            or (self.run_train and self.learn.training)
            or (self.run_valid and not self.learn.training)
        ):
            try:
                output = self.call_event(event_name)
            except (
                CancelBatchException, CancelBackwardException, CancelEpochException, 
                CancelStepException, CancelTrainException, CancelValidatException
            ):
                pass
            except Exception as e:
                raise
        if event_name == "after_fit":
            # Reset self.run to `True` at the end of each `fit`
            self.run = True
        return output